In [1]:
import pandas as pd
import numpy as np

# data processing

In [2]:
# get csv
# df_long = pd.read_csv("data/exp_TMS.csv")


# files = [
#     "data/LIFESPAN_SZcontrols.csv", 
#     "data/LIFESPAN_agingAZ_all.csv", 
#     "data/LIFESPAN_sommerville.csv", 
#     "data/LIFESPAN_students.csv",
#     "data/TMS_horizonTask.csv",
# ]

# files = [
#     "data/exp_TMS.csv", 
#     "data/exp_all.csv", 
# ]
# df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

df_all = pd.read_csv("data/exp_all_filenames_2.csv")
# df_TMS = pd.read_csv("data/exp_TMS.csv")
# df_TMS["file_name"] = "exp_TMS.csv"

# df_TMS['subject'] = df_TMS['subject'].astype(str) + '_TMS'
df_all['subject'] = df_all['subject'].astype(str)
# df = pd.concat([df_all, df_TMS], ignore_index=True)
df = pd.concat([df_all], ignore_index=True)

In [ ]:
df["trial"] = df["trial"].astype(int)

# Rename value_option columns to match original format (m1, m2)
df = df.rename(columns={"value_option0": "m1", "value_option1": "m2"})

# Redefine pivoting function
def pivot_feature(df, feature, prefix):
    return df.pivot_table(
        index=["subject", "block", "m1", "m2", "uncertainty", "horizon"],
        columns="trial",
        values=feature
    ).rename(columns=lambda x: f"{prefix}{x}")

# Pivot reward, choice, RT
reward_wide = pivot_feature(df, "reward", "r")
choice_wide = pivot_feature(df, "choice", "c")
rt_wide = pivot_feature(df, "RT", "rt")

# Get static columns and merge all together
df_static = df.drop_duplicates(subset=["subject", "block", "m1", "m2", "uncertainty", "horizon"])
df_static = df_static.set_index(["subject", "block", "m1", "m2", "uncertainty", "horizon"])
df_wide = pd.concat([df_static, reward_wide, choice_wide, rt_wide], axis=1).reset_index()

df_wide.columns

Index(['subject', 'block', 'm1', 'm2', 'uncertainty', 'horizon', 'age',
       'gender', 'file_name', 'trial', 'reward', 'RT', 'choice', 'forced',
       'mask', 'r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'rt0',
       'rt1', 'rt2', 'rt3', 'rt4', 'rt5', 'rt6', 'rt7', 'rt8', 'rt9'],
      dtype='object')

In [4]:
df_wide

,subject,block,m1,m2,uncertainty,horizon,age,gender,file_name,trial,...,rt0,rt1,rt2,rt3,rt4,rt5,rt6,rt7,rt8,rt9
0,0,0,40,36,1,6,18,-99,LIFESPAN_students.csv,0,...,184.143661,4.635614,1.122532,2.833540,2.738855,5.263696,5.763985,1.940346,0.937745,2.472841
1,0,1,60,48,0,6,18,-99,LIFESPAN_students.csv,0,...,1.184903,1.013671,1.305989,8.874533,2.708196,2.426142,1.115581,1.370487,1.103393,1.046302
2,0,2,40,36,0,6,18,-99,LIFESPAN_students.csv,0,...,0.832413,0.468084,0.566725,0.445963,1.810017,1.325654,0.892086,0.780645,1.881181,0.576540
3,0,3,40,48,0,1,18,-99,LIFESPAN_students.csv,0,...,0.830383,0.553779,0.817720,1.686227,0.910277,NaN,NaN,NaN,NaN,NaN
4,0,4,40,10,0,1,18,-99,LIFESPAN_students.csv,0,...,0.663673,1.049479,0.692706,3.581879,4.616839,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169371,1391,155,60,40,1,6,-99,-99,LIFESPAN_students.csv,0,...,0.376815,0.289047,0.266767,0.549945,0.228912,0.129866,0.107832,0.000469,0.000654,0.000568
169372,1391,156,60,48,-1,1,-99,-99,LIFESPAN_students.csv,0,...,0.265641,0.350753,0.289188,0.323415,0.635624,NaN,NaN,NaN,NaN,NaN
169373,1391,157,52,60,-1,6,-99,-99,LIFESPAN_students.csv,0,...,0.191414,0.395403,0.179858,0.376644,0.232391,0.234998,0.071894,0.010124,0.046230,0.087224
169374,1391,158,64,60,1,6,-99,-99,LIFESPAN_students.csv,0,...,0.235090,0.413205,0.378858,0.370644,0.279961,0.115485,0.100133,0.028765,0.000533,0.380628


In [5]:
df = df_wide.rename(columns={"horizon": "gameLength", "uncertainty": "uc"})

df.columns

Index(['subject', 'block', 'm1', 'm2', 'uc', 'gameLength', 'age', 'gender',
       'file_name', 'trial', 'reward', 'RT', 'choice', 'forced', 'mask', 'r0',
       'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'c0', 'c1', 'c2',
       'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'rt0', 'rt1', 'rt2', 'rt3',
       'rt4', 'rt5', 'rt6', 'rt7', 'rt8', 'rt9'],
      dtype='object')

In [6]:
df.to_csv("data/my_horizon_data_all_filenames.csv", index=False)

# demographic

In [7]:
num_subjects = df["subject"].nunique()
print("Number of unique subjects:", num_subjects)

Number of unique subjects: 1392


In [8]:
# import pandas as pd
# import numpy as np

# # ---- Step 0: normalize gender into {male, female, other, unknown} ----
# def normalize_gender(g):
#     if pd.isna(g):
#         return "unknown"

#     # numeric encoding (your current code implies: 0=male, 1=female)
#     if isinstance(g, (int, np.integer, float, np.floating)):
#         if g == 0:
#             return "male"
#         if g == 1:
#             return "female"
#         return "other"

#     # string encoding (if any files store text labels)
#     s = str(g).strip().lower()
#     if s in {"0", "m", "male", "man"}:
#         return "male"
#     elif s in {"1", "f", "female", "woman"}:
#         return "female"
#     else:
#         return "other"
    
# df = df.copy()
# df["gender_norm"] = df["gender"].apply(normalize_gender)

# # ---- Step 1: collapse to one row per subject per file ----
# participants = (
#     df.groupby(["file_name", "subject"], as_index=False)
#       .agg(gender=("gender_norm", "first"))
# )

# # ---- Step 2: per-file summary ----
# summary = (
#     participants.groupby("file_name", as_index=False)
#       .agg(
#           total_participants=("subject", "nunique"),
#           num_female=("gender", lambda x: (x == "female").sum()),
#           num_male=("gender",   lambda x: (x == "male").sum()),
#           num_other=("gender",  lambda x: (x == "other").sum()),
#         #   num_unknown=("gender",lambda x: (x == "unknown").sum()),
#       )
# )

# # ---- Step 3: compute TOTAL row ----
# total_row = pd.DataFrame({
#     "file_name": ["TOTAL"],
#     "total_participants": [summary["total_participants"].sum()],
#     "num_female": [summary["num_female"].sum()],
#     "num_male": [summary["num_male"].sum()],
#     "num_other": [summary["num_other"].sum()],
#     # "num_unknown": [summary["num_unknown"].sum()],
# })

# # ---- Step 4: append TOTAL row ----
# summary_with_total = pd.concat([summary, total_row], ignore_index=True)

# print(summary_with_total)




In [9]:
# valid age means age != -99
# valid gender means gender is 0 or 1
# valid_subjects = df[
#     (df["age"] != -99) &
#     (df["gender"].isin([0, 1]))
# ]["subject"].unique()
# df_clean = df[df["subject"].isin(valid_subjects)].copy()

In [10]:
import pandas as pd
import numpy as np

# ... (Load 'df' as before from your CSV files) ...

# 1. Define the mapping with \citet{} formatting
# Note: We group BATTERY and AgingAZ into one key.
file_to_paper_map = {
    # Combined Group
    "BATTERY_HorizonTaskFinalFinal.csv": r"\citet{Phelps2025, Mizell2024}",
    "LIFESPAN_agingAZ_all.csv":          r"\citet{Phelps2025, Mizell2024}",
    
    # Individual Groups
    "LIFESPAN_Harms.csv":       r"\citet{Harms2024}",
    "LIFESPAN_SZcontrols.csv":  r"\citet{Waltz2020}",
    "LIFESPAN_Smith.csv":       r"\citet{Smith2022}",
    "LIFESPAN_sommerville.csv": r"\citet{Somerville2017, Wilson2014}",
    "LIFESPAN_students.csv":    r"\citet{Sadeghiyeh2020}"
}

# 2. Define Location Map (Handling the mixed location for the combined group)
paper_to_location_map = {
    r"\citet{Phelps2025, Mizell2024}": r"\begin{tabular}{@{}l@{}}Baltimore, MD\\Tucson, AZ\end{tabular}",
    r"\citet{Harms2024}":              "Madison, WI",
    r"\citet{Waltz2020}":              "Baltimore, MD",
    r"\citet{Smith2022}":              "Tucson, AZ",
    r"\citet{Somerville2017, Wilson2014}": r"\begin{tabular}{@{}l@{}}Cambridge, MA\\Princeton, NJ\end{tabular}",
    r"\citet{Sadeghiyeh2020}":         "Tucson, AZ",
    r"\textbf{Total}":                 "--"
}

# Apply Mapping
df["paper_label"] = df["file_name"].map(file_to_paper_map)

# 3. Collapse to one row per subject
participants_clean = (
    df.groupby(["paper_label", "subject"], as_index=False)
    .agg(
        age=("age", "first"),
        gender=("gender", "first")
    )
)

# 4. Normalize Data
participants_clean["age"] = pd.to_numeric(participants_clean["age"], errors="coerce")
participants_clean["gender_cat"] = np.select(
    [participants_clean["gender"] == 1, participants_clean["gender"] == 0],
    ["female", "male"],
    default="other"
)
participants_clean["gender_cat"] = pd.Categorical(participants_clean["gender_cat"], categories=["female", "male", "other"])

# 5. Assign Age Groups
is_unknown = (participants_clean["age"] < 0) | participants_clean["age"].isna()
conds = [
    is_unknown,
    participants_clean["age"] < 16,
    (participants_clean["age"] >= 17) & (participants_clean["age"] <= 32),
    (participants_clean["age"] >= 33) & (participants_clean["age"] <= 59),
    participants_clean["age"] >= 60
]
choices = ["Unknown", "<16", "17-32", "33-59", ">60"]
participants_clean["age_group"] = np.select(conds, choices, default="Unknown")
participants_clean["age_group"] = pd.Categorical(
    participants_clean["age_group"], 
    categories=["<16", "17-32", "33-59", ">60", "Unknown"], 
    ordered=True
)

# 6. Calculate Stats by Paper
paper_summary = (
    participants_clean
    .groupby(["paper_label", "age_group"], observed=False)
    .agg(
        n=("subject", "nunique"),
        num_female=("gender_cat", lambda x: (x == "female").sum()),
        num_male=("gender_cat", lambda x: (x == "male").sum()),
        num_other=("gender_cat", lambda x: (x == "other").sum()),
        age_min=("age", "min"),
        age_max=("age", "max"),
        age_mean=("age", "mean"),
        age_sd=("age", "std"),
    )
    .reset_index()
)
paper_summary = paper_summary[paper_summary["n"] > 0].copy()

# 7. Add Location Column
paper_summary["location"] = paper_summary["paper_label"].map(paper_to_location_map)

# 8. Add Total Row
total_row = (
    participants_clean
    .groupby("age_group", observed=True)
    .agg(
        n=("subject", "nunique"),
        num_female=("gender_cat", lambda x: (x == "female").sum()),
        num_male=("gender_cat", lambda x: (x == "male").sum()),
        num_other=("gender_cat", lambda x: (x == "other").sum()),
        age_min=("age", "min"),
        age_max=("age", "max"),
        age_mean=("age", "mean"),
        age_sd=("age", "std"),
    )
    .reset_index()
)
total_row.insert(0, "paper_label", r"\textbf{Total}")
total_row["location"] = "--"

# Combine
final_df = pd.concat([paper_summary, total_row], ignore_index=True)

# 9. Format "Unknown" stats
stat_cols = ["age_min", "age_max", "age_mean", "age_sd"]
final_df[stat_cols] = final_df[stat_cols].astype(object)
mask = final_df["age_group"] == "Unknown"
final_df.loc[mask, stat_cols] = "--"

print(final_df)

                           paper_label age_group    n  num_female  num_male  \
0                    \citet{Harms2024}       <16   53        25.0      28.0   
1                    \citet{Harms2024}     17-32   58        14.0      44.0   
2                    \citet{Harms2024}     33-59    2         1.0       1.0   
3                    \citet{Harms2024}   Unknown   10         0.0       0.0   
4       \citet{Phelps2025, Mizell2024}     17-32  151        84.0      65.0   
5       \citet{Phelps2025, Mizell2024}       >60  198       117.0      80.0   
6               \citet{Sadeghiyeh2020}     17-32  346       108.0     237.0   
7               \citet{Sadeghiyeh2020}     33-59    1         1.0       0.0   
8               \citet{Sadeghiyeh2020}   Unknown   22         2.0       8.0   
9                    \citet{Smith2022}     17-32  382       278.0     104.0   
10                   \citet{Smith2022}     33-59   39        26.0      13.0   
11  \citet{Somerville2017, Wilson2014}       <16   5

In [11]:
# import pandas as pd

def generate_final_latex():
    # Define the data exactly as in your provided table
    # calculated 'total_n' by summing 'n' for each paper group
    data = [
        {
            "paper": r"\citet{Harms2024}", 
            "total_n": 123, 
            "loc": "Madison, WI",
            "rows": [
                {"group": "Teens", "n": 53, "f": 25, "m": 28, "o": 0, "mean": 11.21, "sd": 0.77},
                {"group": "Young", "n": 58, "f": 14, "m": 44, "o": 0, "mean": 19.29, "sd": 2.63},
                {"group": "Middle", "n": 2, "f": 1, "m": 1, "o": 0, "mean": 37.5, "sd": 0.71},
                {"group": "Unknown", "n": 10, "f": 0, "m": 0, "o": 10, "mean": "--", "sd": "--"},
            ]
        },
        {
            "paper": r"\begin{tabular}{@{}l@{}}\citet{Phelps2025}\\\citet{Mizell2024}\end{tabular}", 
            "total_n": 349, 
            "loc": r"\begin{tabular}{@{}l@{}}Tucson, AZ\end{tabular}",
            "rows": [
                {"group": "Young", "n": 151, "f": 84, "m": 65, "o": 2, "mean": 21.43, "sd": 2.87},
                {"group": "Old", "n": 198, "f": 117, "m": 80, "o": 1, "mean": 69.17, "sd": 2.71},
            ]
        },
        {
            "paper": r"\citet{Sadeghiyeh2020}", 
            "total_n": 369, 
            "loc": "Tucson, AZ",
            "rows": [
                {"group": "Young", "n": 346, "f": 108, "m": 237, "o": 1, "mean": 19.42, "sd": 1.59},
                {"group": "Middle", "n": 1, "f": 1, "m": 0, "o": 0, "mean": 50.0, "sd": "--"},
                {"group": "Unknown", "n": 22, "f": 2, "m": 8, "o": 12, "mean": "--", "sd": "--"},
            ]
        },
        {
            "paper": r"\citet{Smith2022}", 
            "total_n": 421, 
            "loc": "Tucson, AZ",
            "rows": [
                {"group": "Young", "n": 382, "f": 278, "m": 104, "o": 0, "mean": 22.41, "sd": 3.94},
                {"group": "Middle", "n": 39, "f": 26, "m": 13, "o": 0, "mean": 36.44, "sd": 2.29},
            ]
        },
        {
            "paper": r"\begin{tabular}{@{}l@{}}\citet{Somerville2017}\\\citet{Wilson2014}\end{tabular}", 
            "total_n": 92, 
            "loc": r"\begin{tabular}{@{}l@{}}Cambridge, MA\\Princeton, NJ\end{tabular}",
            "rows": [
                {"group": "Teens", "n": 50, "f": 24, "m": 26, "o": 0, "mean": 13.42, "sd": 1.01},
                {"group": "Young", "n": 22, "f": 13, "m": 9, "o": 0, "mean": 17.0, "sd": 0.0},
                {"group": "Unknown", "n": 20, "f": 10, "m": 10, "o": 0, "mean": "--", "sd": "--"},
            ]
        },
        {
            "paper": r"\citet{Waltz2020}", 
            "total_n": 38, 
            "loc": "Baltimore, MD",
            "rows": [
                {"group": "Young", "n": 16, "f": 5, "m": 11, "o": 0, "mean": 25.94, "sd": 4.04},
                {"group": "Middle", "n": 20, "f": 6, "m": 14, "o": 0, "mean": 44.2, "sd": 5.63},
                {"group": "Unknown", "n": 2, "f": 0, "m": 0, "o": 2, "mean": "--", "sd": "--"},
            ]
        },
        {
            "paper": r"\textbf{Total}", 
            "total_n": 1392, 
            "loc": "--",
            "rows": [
                {"group": "Teens", "n": 103, "f": 49, "m": 54, "o": 0, "mean": 12.28, "sd": 1.42},
                {"group": "Young", "n": 975, "f": 502, "m": 470, "o": 3, "mean": 20.94, "sd": 3.4},
                {"group": "Middle", "n": 62, "f": 34, "m": 28, "o": 0, "mean": 39.19, "sd": 5.31},
                {"group": "Old", "n": 198, "f": 117, "m": 80, "o": 1, "mean": 69.17, "sd": 2.71},
                {"group": "Unknown", "n": 54, "f": 12, "m": 18, "o": 24, "mean": "--", "sd": "--"},
                # Grand Total Row
                {"group": r"\textbf{All}", "n": 1392, "f": 714, "m": 650, "o": 28, "mean": "--", "sd": "--", "is_bold": True},
            ]
        }
    ]

    latex = []
    latex.append(r"\begin{table}[h!]")
    latex.append(r"\caption{Participant demographics across datasets stratified by age group}\label{tab:demographic_detailed}")
    
    # Updated column structure: 10 columns
    # Paper, Total N, Group, F, M, O, n(Group), Mean, SD, Location
    latex.append(r"\begin{tabular*}{\textwidth}{@{\extracolsep\fill}lclcccccccl}")
    latex.append(r"\toprule")
    
    # Header Row 1
    latex.append(r"& \multicolumn{1}{c}{Total} & & \multicolumn{3}{c}{Gender} & \multicolumn{3}{c}{Age} & \multicolumn{1}{c}{Location} \\")
    
    # Header Rules
    latex.append(r"\cmidrule(lr){2-2}\cmidrule(lr){4-6}\cmidrule(lr){7-9}\cmidrule(lr){10-10}")
    
    # Header Row 2
    latex.append(r"Paper & $N$ & Group & F & M & O & $n$ & Mean & SD & City, State \\")
    latex.append(r"\midrule")

    for section in data:
        rows = section["rows"]
        num_rows = len(rows)
        paper_label = section["paper"]
        total_n = section["total_n"]
        loc_label = section["loc"]
        
        # Determine styling for Total N column (bold if it's the Total section)
        if paper_label == r"\textbf{Total}":
             # For the Total section, we might leave the Paper N empty or match the sum
             # Usually standard to put the total N at the top or leave it blank if redundant
             # Here we will display it as multirow
             total_n_str = f"\\textbf{{{total_n}}}"
        else:
             total_n_str = str(total_n)

        first = True
        for i, row in enumerate(rows):
            # Columns 1 & 2 & 10: Multirow Logic
            if first:
                if paper_label == r"\textbf{Total}":
                     # Total section: Location is --, Paper is Total
                     p_col = f"\\multirow{{{num_rows}}}{{*}}{{{paper_label}}}"
                     tn_col = f"\\multirow{{{num_rows}}}{{*}}{{{total_n_str}}}"
                     l_col = f"\\multirow{{{num_rows}}}{{*}}{{--}}"
                else:
                     p_col = f"\\multirow{{{num_rows}}}{{*}}{{{paper_label}}}"
                     tn_col = f"\\multirow{{{num_rows}}}{{*}}{{{total_n_str}}}"
                     l_col = f"\\multirow{{{num_rows}}}{{*}}{{{loc_label}}}"
            else:
                p_col = ""
                tn_col = ""
                l_col = ""

            # Data Columns
            group = row['group']
            n = row['n']
            f = row['f']
            m = row['m']
            o = row['o']
            mean = row['mean']
            sd = row['sd']

            # Bold Row Logic (for the 'All' row)
            if row.get("is_bold"):
                n = f"\\textbf{{{n}}}"
                f = f"\\textbf{{{f}}}"
                m = f"\\textbf{{{m}}}"
                o = f"\\textbf{{{o}}}"
                mean = r"\textbf{--}"
                sd = r"\textbf{--}"
                # l_col is already handled via multirow or empty, but for the last row usually we leave it
                # If 'All' is the last row of 'Total' section, it falls into the multirow block.
            
            line = f"{p_col} & {tn_col} & {group} & {f} & {m} & {o} & {n} & {mean} & {sd} & {l_col} \\\\"
            latex.append(line)
            first = False

        if section != data[-1]:
            latex.append(r"\cmidrule(lr){1-10}")

    latex.append(r"\botrule")
    latex.append(r"\end{tabular*}")
    latex.append(r"\end{table}")

    return "\n".join(latex)

print(generate_final_latex())

\begin{table}[h!]
\caption{Participant demographics across datasets stratified by age group}\label{tab:demographic_detailed}
\begin{tabular*}{\textwidth}{@{\extracolsep\fill}lclcccccccl}
\toprule
& \multicolumn{1}{c}{Total} & & \multicolumn{3}{c}{Gender} & \multicolumn{3}{c}{Age} & \multicolumn{1}{c}{Location} \\
\cmidrule(lr){2-2}\cmidrule(lr){4-6}\cmidrule(lr){7-9}\cmidrule(lr){10-10}
Paper & $N$ & Group & F & M & O & $n$ & Mean & SD & City, State \\
\midrule
\multirow{4}{*}{\citet{Harms2024}} & \multirow{4}{*}{123} & Teens & 25 & 28 & 0 & 53 & 11.21 & 0.77 & \multirow{4}{*}{Madison, WI} \\
 &  & Young & 14 & 44 & 0 & 58 & 19.29 & 2.63 &  \\
 &  & Middle & 1 & 1 & 0 & 2 & 37.5 & 0.71 &  \\
 &  & Unknown & 0 & 0 & 10 & 10 & -- & -- &  \\
\cmidrule(lr){1-10}
\multirow{2}{*}{\begin{tabular}{@{}l@{}}\citet{Phelps2025}\\\citet{Mizell2024}\end{tabular}} & \multirow{2}{*}{349} & Young & 84 & 65 & 2 & 151 & 21.43 & 2.87 & \multirow{2}{*}{\begin{tabular}{@{}l@{}}Tucson, AZ\end{tabular}} \\
 &